In [1]:
import sys
import pandas as pd
import numpy as np
import random
import json, re

from PyQt6.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QWidget, QComboBox, QCompleter, QLineEdit,  QVBoxLayout,  QWidget, QSizePolicy, QDockWidget
from PyQt6.QtCore import pyqtSignal, Qt
from PyQt6.QtGui import *


## Read Files
with open('output.json', 'r') as file:
    combos = json.load(file)

clustered = pd.read_csv("C:\\Users\\Admin\\Downloads\\clusters.csv")

df = pd.read_csv("C:\\Users\\Admin\\Downloads\\ABBREVcsv.csv")


## Average nutrient profile per cluster
nutrients = clustered[['Calcium_(mg)', 'Iron_(mg)', 'Magnesium_(mg)',
       'Phosphorus_(mg)', 'Potassium_(mg)', 'Zinc_(mg)',
       'Copper_mg)', 'Manganese_(mg)', 'Selenium_(mcg)', 'Vit_C_(mg)',
       'Thiamin_(mg)', 'Riboflavin_(mg)', 'Niacin_(mg)', 'Panto_Acid_mg)',
       'Vit_B6_(mg)', 'Folate_Tot_(mcg)','Choline_Tot_ (mg)',
       'Vit_B12_(mcg)', 'Vit_A_RAE', 'Vit_E_(mg)', 'Vit_D_mcg',
       'Vit_K_(mcg)',"micro"]]

clustermeans = nutrients.groupby(["micro"]).mean()

class MainWindow(QMainWindow):
    
    def __init__(self):
        super().__init__()
        self.resize(500, 500)
        self.setWindowTitle("Nutrition Nuts")
        self.setStyleSheet("background-color: #38761D; color: white;")        

        QFontDatabase.addApplicationFont("C:\\Users\\Admin\\Downloads\\BebasNeue-Regular.ttf") # Change Path
        QFontDatabase.addApplicationFont("C:\\Users\\Admin\\Downloads\\Roboto-Thin.ttf")
        QFontDatabase.addApplicationFont("C:\\Users\\Admin\\Downloads\\Roboto-Light.ttf")
        QFontDatabase.addApplicationFont("C:\\Users\\Admin\\Downloads\\Roboto-Regular.ttf")

        layout = QVBoxLayout()

        self.label = QLabel("Nutrition Nuts", self)
        self.label.setFont(QFont('Bebas Neue', 50))
        self.label.move(160, 300)
        self.label.resize(200, 200)
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter) 
        self.label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding) # Maintain proportions

        pic = QLabel()
        pixmap = QPixmap("C:\\Users\Admin\\Downloads\\dish.png") # Change Path
        pic.setPixmap(pixmap)
        pic.setAlignment(Qt.AlignmentFlag.AlignCenter) 
       
        self.button = QPushButton("Start", self)
        self.button.setStyleSheet("background-color: transparent; border: none;")  
        self.button.setStyleSheet("text-align: center;")
        self.button.setFont(QFont('Roboto', 20))
        self.button.setGeometry(150, 200, 150, 30)
        self.button.clicked.connect(self.open_new_window)

        self.label_1 = QLabel("\n \n By: Amaris Williams, Madelyn Cruz, Jakeah Phifer \n \n", self)
        self.label_1.setFont(QFont('Roboto', 10))
        self.label_1.move(20, 100)
        self.label_1.resize(60, 60)
        self.label_1.setAlignment(Qt.AlignmentFlag.AlignBottom)               

        layout.addWidget(self.label)
        layout.addWidget(pic)
        layout.addWidget(self.label_1)        
        layout.addWidget(self.button)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)
    
    def open_new_window(self):
        self.new_window = UserInputFood()
        self.new_window.show()
        self.new_window.closed.connect(self.close)
        self.hide()
  

class UserInputFood(QMainWindow):

    closed = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.resize(500, 500)
        self.setWindowTitle("Nutrition Nuts")
        self.setStyleSheet("background-color: #93C47D;  color: white;")        

        layout = QVBoxLayout()

        self.label = QLabel("Input food item to be matched", self)
        self.label.setFont(QFont('Bebas Neue', 20))
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter) 
        self.label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)           
        
        common_foods = QCompleter( [word.title() for word in clustered['Shrt_Desc'].values] )
        self.fruit = QLineEdit(self)
        self.fruit.setCompleter(common_foods)
        self.fruit.setGeometry(100, 100, 250, 30)

        self.button = QPushButton("Next", self)
        self.button.setStyleSheet("background-color: transparent; border: none;")  
        self.button.setStyleSheet("text-align: center;")
        self.button.setFont(QFont('Bebas Neue', 20))
        self.button.setGeometry(100, 100, 250, 30)
        self.button.clicked.connect(self.display) 

        layout.addWidget(self.label)     
        layout.addWidget(self.fruit)   
        layout.addWidget(self.button)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)
       
    def display(self):
        text = self.fruit.text()
        print(text)        
        #print(any(self.fruit.text().lower() in i for i in [word.title().lower() for word in clustered['Shrt_Desc'].values]) )

        if any(self.fruit.text().lower() in i for i in [word.title().lower() for word in clustered['Shrt_Desc'].values]): # If not in list, find complete words that is most popular
            cc = clustered[clustered['Shrt_Desc'].str.contains(self.fruit.text(), case=False,regex=False)]
            self.food_to_match = cc.loc[[cc['popularity'].idxmax()]].Shrt_Desc.values[0]
            microgroup = clustered.iloc[clustered[clustered['Shrt_Desc'].str.contains(self.food_to_match, case=False, regex=False)].index].micro.values[0]
            microgroup_index = clustered[clustered['Shrt_Desc'].str.contains(self.food_to_match, case=False, regex=False)].index
            
            index_array = clustermeans.index.to_numpy()
            i = np.where(index_array == clustered.iloc[microgroup_index].micro.values[0])[0][0]

            matching_lists = [lst for lst in combos if i in lst]
            random_list = random.choice(matching_lists)
            random_list.remove(i)

            self.foods_to_suggest = []
            for pp in random_list:
                aa = clustered[clustered["micro"] ==clustermeans.index[pp]]
                self.foods_to_suggest.append(aa.nlargest(10, 'popularity').Shrt_Desc.values.tolist())

            #print(self.foods_to_suggest)
            #print(microgroup)
            self.button.clicked.connect(self.open_new_window)
            
        else:
            self.new_window = UserInputFood()
            self.new_window.show()
            self.new_window.closed.connect(self.close)
            self.hide()
        
    def open_new_window(self):        
        self.new_window = Recommendation1(self.food_to_match, self.foods_to_suggest)
        self.new_window.show()
        self.new_window.closed.connect(self.close)
        self.hide()

class Recommendation1(QMainWindow):

    closed = pyqtSignal()
    def __init__(self, fruit, foods_to_suggest):
        super().__init__()
        self.resize(500, 500)
        self.setWindowTitle("Nutrition Nuts: Recommendation")
        self.setStyleSheet("background-color: #93C47D;  color: white;")        

        layout = QVBoxLayout()

        self.fruit = fruit
        self.foods_to_suggest = foods_to_suggest      
        
        self.label = QLabel(self)
        formatted_fruit = re.sub(r',(?!\s)', ', ', fruit.title())
        self.label.setText(f"Match your \n {formatted_fruit} \n with:")
        self.label.setFont(QFont('Bebas Neue', 20))
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter) 
        self.label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)

        self.button = QPushButton("Customize Recommendation", self)
        self.button.setStyleSheet("background-color: transparent; border: none;")  
        self.button.setStyleSheet("text-align: center;")
        self.button.setFont(QFont('Bebas Neue', 20))
        self.button.setGeometry(100, 100, 250, 30)
        #self.button.clicked.connect(self.display)
        self.button.clicked.connect(self.open_new_window)

        


        

        label2 = QLabel()
        html = "<ul>"
        for item in self.foods_to_suggest:
            toprint = item[0].title()
            html += "<li>{}</li>".format(re.sub(r',(?!\s)', ', ', toprint))    
        html += "</ul><br><br><br><br>"
        label2.setText(html)        
        label2.setOpenExternalLinks(True)
        label2.setFont(QFont('Roboto', 12))
        label2.setAlignment(Qt.AlignmentFlag.AlignCenter) 

        central_widget = QWidget()
        central_widget.setLayout(layout)

        layout.addWidget(self.label)
        layout.addWidget(label2)       
        layout.addWidget(self.button)
        self.setCentralWidget(central_widget)
 
    def open_new_window(self):
        self.new_window = Recommendation2(self.fruit, self.foods_to_suggest)
        self.new_window.show()
        self.new_window.closed.connect(self.close)
        self.hide()



class Recommendation2(QMainWindow):

    closed = pyqtSignal()
    def __init__(self, fruit, foods_to_suggest):
        super().__init__()
        self.resize(500, 500)
        self.setWindowTitle("Nutrition Nuts: Recommendation")
        self.setStyleSheet("background-color: #93C47D;  color: white;")        

        layout = QVBoxLayout()

        self.fruit = fruit
        self.foods_to_suggest = foods_to_suggest
      
        self.label = QLabel(self)
        layout.addWidget(self.label)
        formatted_fruit = re.sub(r',(?!\s)', ', ', fruit.title())
        self.label.setText(f"Match your \n {formatted_fruit} \n with:")
        self.label.setFont(QFont('Bebas Neue', 20))
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter) 
        #Everytime you change one food in the recommended, the dropdown options should change too.
        
        self.label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)
             
        def get_combobox_values(combo_boxes):
            values = []
            for combo_box in combo_boxes:
                current_value = combo_box.currentText()
                values.append(current_value)
            return values
        
        layout.addWidget(self.label)     

        combos = []
        for i in range(len(self.foods_to_suggest)):
            self.combo = QComboBox(self)
            self.combo.addItems(self.foods_to_suggest[i])

            combos.append(self.combo)
            #self.combo.activated.connect(self.submit_selection)
            self.combo.move(100,200+30*i)
            layout.addWidget(self.combo)

        self.final_foods = get_combobox_values(combos)

        self.button = QPushButton("Next", self)
        self.button.setStyleSheet("background-color: transparent; border: none;")  
        self.button.setStyleSheet("text-align: center;")
        self.button.setFont(QFont('Bebas Neue', 20))
        self.button.setGeometry(100, 100, 250, 30)
        self.button.clicked.connect(self.open_new_window)
        
        label2 = QLabel()
        label2.setText("<br><br><br><br><br><br>")
        label2.setOpenExternalLinks(True)
        label2.setFont(QFont('Roboto', 12))
        label2.setAlignment(Qt.AlignmentFlag.AlignCenter) 

        central_widget = QWidget()
        central_widget.setLayout(layout)

        layout.addWidget(label2)       
        layout.addWidget(self.button)
        self.setCentralWidget(central_widget)   
 
    def open_new_window(self):       
        print("TO MATCH", self.fruit)
        print("FINAL", self.final_foods)
        self.new_window = FinalRecommendation(self.fruit , self.final_foods)
        self.new_window.show()
        self.new_window.closed.connect(self.close)
        self.hide()

   # def submit_selection(self):
    #    print("TO MATCH", self.fruit)
      #  print("CURRENT SELECTIONS", self.final_foods)
#
       # foods_to_suggest = [['X 1', 'X 2', 'X 3'], ['Y 1', 'Y 2', 'Y 3'], ['Z 1', 'Z 2', 'Z 3', 'Z 4', 'Z 5']]

       # self.new_window = Recommendation2(self.fruit, foods_to_suggest)
       # self.new_window.show()
       # self.new_window.closed.connect(self.close)
       # self.hide()
        
class FinalRecommendation(QMainWindow):
    closed = pyqtSignal()
    def __init__(self, fruit, final_foods):
        super().__init__()
        self.resize(500, 500)
        self.setWindowTitle("Nutrition Nuts")
        self.setStyleSheet("background-color: #93C47D;  color: white;")        

        layout = QVBoxLayout()

        self.fruit = fruit
        self.final_foods = final_foods      

        self.label = QLabel(self)
        self.label.setText(f"Your balanced meal for today: ")
        self.label.setFont(QFont('Bebas Neue', 20))
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)         
        self.label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)
           
        def on_quit():
            app.quit()

        self.button = QPushButton("Exit", self)
        self.button.setStyleSheet("background-color: transparent; border: none;")  
        self.button.setStyleSheet("text-align: center;")
        self.button.setFont(QFont('Bebas Neue', 20))
        self.button.setGeometry(100, 100, 250, 30)
        self.button.clicked.connect(on_quit)        
        
        self.button2 = QPushButton("More Balanced Meals", self)
        self.button2.setStyleSheet("background-color: transparent; border: none;")  
        self.button2.setStyleSheet("text-align: center;")
        self.button2.setFont(QFont('Bebas Neue', 20))
        self.button2.setGeometry(100, 100, 250, 30)
        self.button2.clicked.connect(self.other_combinations)     

        
        label2 = QLabel()

        meas = df [df.Shrt_Desc == self.fruit].GmWt_1.values[0]
        measunit = df[df .Shrt_Desc == self.fruit].GmWt_Desc1.values[0]

        toprint = str(meas) + " x " + str(measunit)  + " " + fruit.title()
        html = "<ul>"
        html += "<li>{}</li>".format(re.sub(r',(?!\s)', ', ', toprint ))

        for item in self.final_foods:         
            print("item")
            print(item)   
            meas = df[df.Shrt_Desc == item].GmWt_1.values[0]
            measunit = df[df .Shrt_Desc == item].GmWt_Desc1.values[0]

        
            toprint = str(meas) + " x " + str(measunit) + " "   + item.title()
            html += "<li>{}</li>".format(re.sub(r',(?!\s)', ', ', toprint))   
        html += "</ul><br><br><br><br><br><br>"

        label2.setText(html)
        label2.setOpenExternalLinks(True)
        label2.setFont(QFont('Roboto', 12))
        label2.setAlignment(Qt.AlignmentFlag.AlignCenter) 

        central_widget = QWidget()
        central_widget.setLayout(layout)

        layout.addWidget(self.label)   
        layout.addWidget(label2)        
        layout.addWidget(self.button2)       
        layout.addWidget(self.button)
        self.setCentralWidget(central_widget)
    
    def other_combinations(self):
        microgroup = clustered.iloc[clustered[clustered['Shrt_Desc'].str.contains(self.fruit, case=False, regex=False)].index].micro.values[0]
        microgroup_index = clustered[clustered['Shrt_Desc'].str.contains(self.fruit, case=False, regex=False)].index
                
        index_array = clustermeans.index.to_numpy()
        i = np.where(index_array == clustered.iloc[microgroup_index].micro.values[0])[0][0]

        matching_lists = [lst for lst in combos if i in lst]
        random_list = random.choice(matching_lists)
        random_list.remove(i)
        random_list

        self.foods_to_suggest = []
        for pp in random_list:
            aa = clustered[clustered["micro"] ==clustermeans.index[pp]]
            self.foods_to_suggest.append(aa.nlargest(10, 'popularity').Shrt_Desc.values.tolist())

        print(self.foods_to_suggest)

        print(microgroup)
        self.new_window = Recommendation1(self.fruit, self.foods_to_suggest)
        self.new_window.show()
        self.new_window.closed.connect(self.close)
        self.hide()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec())


butter
butter
TO MATCH BUTTER,WITH SALT
FINAL ['LENTILS,MATURE SEEDS,CKD,BLD,WO/SALT', 'RESTAURANT,CHINESE,ORANGE CHICK', 'SARDINE,ATLANTIC,CND IN OIL,DRND SOL W/BONE']
item
LENTILS,MATURE SEEDS,CKD,BLD,WO/SALT
item
RESTAURANT,CHINESE,ORANGE CHICK
item
SARDINE,ATLANTIC,CND IN OIL,DRND SOL W/BONE


In [1]:
import sys
import pandas as pd
import numpy as np
import random
import json, re

In [13]:
measA

133.0